In [ ]:
import pandas as pd
# from scipy.stats import pearsonr

#create DataFrame
df = pd.read_csv('2022-jd.csv')
highest = {}

cols = df.columns
cols = cols.drop('date').drop('bottles drank').drop('times peed').drop('time slept').drop('time awaked').drop('total sleep time').drop('effectiveness (Am I SPENDING the RIGHT TIME)').drop('Sleep Min').drop('Sleep Hr').drop('Awake Hr').drop('% of planned output')


# calculate correlation coefficient between all cols
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        col1 = cols[i]
        col2 = cols[j]
        # try:
        # corr, p = pearsonr(df[col1].dropna(), df[col2].dropna())
        # except:
        corr = df[col1].corr(df[col2])
        # p=None
        highest[col1 + '-' + col2] = f'{corr}'#, {p}'

highest = sorted(highest.items(), key=lambda x: x[1], reverse=True)
for k, v in highest:
    print(k, v, sep=':\t')

In [ ]:
#create DataFrame
df = pd.read_csv('2022-jd.csv')
highest = {}

cols = df.columns
cols = cols.drop('date').drop('bottles drank').drop('times peed').drop('time slept').drop('time awaked').drop('total sleep time').drop('effectiveness (Am I SPENDING the RIGHT TIME)').drop('Sleep Min').drop('Sleep Hr').drop('Awake Hr').drop('% of planned output')


# calculate correlation coefficient between all cols
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        col1 = cols[i]
        col2 = cols[j]
        # try:
        # corr, p = pearsonr(df[col1].dropna(), df[col2].dropna())
        # except:
        corr = df[col1].corr(df[col2])
        # p=None
        highest[col1 + '-' + col2] = f'{corr}'#, {p}'

highest = sorted(highest.items(), key=lambda x: x[1], reverse=True)
for k, v in highest:
    print(k, v, sep=':\t')

In [ ]:
import pandas as pd
import glob
import os

path = r'/Users/rami/Library/CloudStorage/OneDrive-UniversityofCalgary/code/python/hello_ds/journal-data-analysis/2022-activity' # use your path

all_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# save to csv
df.to_csv('2022-a.csv', index=False, encoding='utf-8')

path = r'/Users/rami/Library/CloudStorage/OneDrive-UniversityofCalgary/code/python/hello_ds/journal-data-analysis/2022-sleep' # use your path

all_files = glob.glob(os.path.join(path, "*.csv"))
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# save to csv
df.to_csv('2022-s.csv', index=False, encoding='utf-8')

In [ ]:
# DANGEROUS, remove first line of all files in 'all_files'
for f in all_files:
    with open(f, 'r') as fin:
        data = fin.read().splitlines(True)
    with open(f, 'w') as fout:
        fout.writelines(data[1:])

In [ ]:
import pandas as pd

# Read in the two CSV files
df1 = pd.read_csv("2022-s.csv")
df2 = pd.read_csv("2022-a.csv")

# Merge the two dataframes on a common column
merged_df = df2.join(df1.set_index('Date'), on='Date')

# Save the merged dataframe to a new CSV file
merged_df.to_csv("2022-fb.csv", index=False)

In [9]:
with open('2022-m.csv', 'r') as fin:
    data = fin.read().splitlines(True)
    # get the number of times each date was mentioned
    dates = {}
    for date in data:
        date = date.strip()
        if date in dates:
            dates[date] += 1
        else:
            dates[date] = 1

In [5]:

import pandas as pd

# Read in the two CSV files
df1 = pd.read_csv("2022-fb.csv")
df2 = pd.read_csv("2022-jd.csv")

# Merge the two dataframes on a common column
merged_df = df2.join(df1.set_index('Date'), on='Date')

# add the number of times each date was mentioned to its row
merged_df['scrobbles'] = merged_df['Date'].map(dates)

# Save the merged dataframe to a new CSV file
merged_df.to_csv("2022.csv", index=False)

In [ ]:
import pandas as pd

# Read in the two CSV files
df1 = pd.read_csv("2022.csv")
df2 = pd.read_csv("sleepStress.csv")

# Merge the two dataframes on a common column
merged_df = df1.join(df2.set_index('Date'), on='Date')


# Save the merged dataframe to a new CSV file
merged_df.to_csv("2022.csv", index=False)